In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import create_dataset
from torchsummary import summary as summary_

import os
import pandas as pd
import numpy as np

import warnings

warnings.filterwarnings('ignore')


# variable
raw_image_size = 8

### 데이터 로드 및 분리

In [9]:
name_dict = {'cham': 0, 'junguil': 1, 'lee': 2, 'leejea': 3, 'moon': 4, 'shin': 5, 'you': 6}

total_ds = create_dataset.make_total_dataset(8, name_dict)
total_ds_test = create_dataset.make_total_dataset(8, name_dict, is_train=False)

### ResNet18 모델 정의 및 인스턴스 초기화

In [12]:
# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=7):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        #self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(256, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        #out = self.layer4(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
    

# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [5]:
model = ResNet18()
summary_(model, (1,8,8),100)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [100, 64, 8, 8]             576
       BatchNorm2d-2            [100, 64, 8, 8]             128
            Conv2d-3            [100, 64, 8, 8]          36,864
       BatchNorm2d-4            [100, 64, 8, 8]             128
            Conv2d-5            [100, 64, 8, 8]          36,864
       BatchNorm2d-6            [100, 64, 8, 8]             128
        BasicBlock-7            [100, 64, 8, 8]               0
            Conv2d-8            [100, 64, 8, 8]          36,864
       BatchNorm2d-9            [100, 64, 8, 8]             128
           Conv2d-10            [100, 64, 8, 8]          36,864
      BatchNorm2d-11            [100, 64, 8, 8]             128
       BasicBlock-12            [100, 64, 8, 8]               0
           Conv2d-13           [100, 128, 4, 4]          73,728
      BatchNorm2d-14           [100, 12

### 데이터셋(Dataset) 다운로드 및 불러오기

In [13]:
train_loader = torch.utils.data.DataLoader(dataset=total_ds, batch_size=100, num_workers=0, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=total_ds_test, batch_size=100, num_workers=0, shuffle=True)

In [14]:
train_loader

### 환경 설정 및 학습(Training) 함수 정의

In [15]:
device = 'cpu'
#device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.001
file_name = 'resnet18_BDP.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, target1, target2) in enumerate(train_loader): # 데이터, 이름, 자세
        inputs, target1 = inputs.to(device), target1.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)#.type(torch.float32)
        #benign_outputs = torch.tensor(benign_outputs, dtype=torch.float32)
        
        loss = criterion(benign_outputs, target1)
        #loss.requires_grad = True
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += target1.size(0)
        correct += predicted.eq(target1).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(target1).sum().item() / target1.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, target1, target2) in enumerate(test_loader):
        inputs, target1 = inputs.to(device), target1.to(device)
        total += target1.size(0)

        outputs = net(inputs)#.type(torch.float32)
        loss += criterion(outputs, target1).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(target1).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 5:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

### 학습(Training) 진행

In [16]:
for epoch in range(0, 10):
    #adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.15
Current benign train loss: 2.1253440380096436

Total benign train accuarcy: 86.25710227272727
Total benign train loss: 18.712525375187397

[ Test epoch: 0 ]

Test accuarcy: 14.204545454545455
Test average loss: 0.024549408392472702
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.08051031082868576

Total benign train accuarcy: 99.96448863636364
Total benign train loss: 1.115032048895955

[ Test epoch: 1 ]

Test accuarcy: 59.09090909090909
Test average loss: 0.017273022369904953
Model Saved!

[ Train epoch: 2 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.018503380939364433

Total benign train accuarcy: 100.0
Total benign train loss: 0.4741390310227871

[ Test epoch: 2 ]

Test accuarcy: 59.94318181818182
Test average loss: 0.013131861998276277
Model Saved!

[ Train epoch: 3 ]

Current batch: 0
Current benign trai